## Cuaderno para ejecutar telegram-tracker-t-hoarder_tg

### Montamos nuestro drive en Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalamos telethon

In [2]:
!python3 -m pip install --upgrade telethon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.8/599.8 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for telethon: filename=Telethon-1.35.0-py3-none-any.whl size=675107 sha256=8f33d3aaa4572aabe0c6cc57b39ebe9344882616e6078d51410f20fb4d30f8f1
  Stored in directory: /root/.cache/pip/wheels/7b/18/81/4a5a3adfd4af392db743a5a5942157c220bbedda57a17d6bbf
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=45379478e122cfcfd9752d49ea50ebc13c42a613a2ecec4db2153dad3d1c4c02
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built telethon pyaes


### Ejecutamos el script principal de telegram-tracket



```
usage: main.py
       [-h]
       --telegram-channel
       TELEGRAM_CHANNEL
       [--limit-download-to-channel-metadata]
       [--output OUTPUT]


Opciones:
 --telegram-channel  para descargar un canal
 --limit-download-to-channel-metadata bajar solo los metadatos del canal (sin mensajes)
 --output directorio donde descargar los datos
```



### Extraer un canal

Será la raiz de la búsqueda por bola de nieve


In [4]:
import os
# Establecer el directorio de trabajo
os.chdir("/content/drive/MyDrive/telegram-tracker-t-hoarder_tg-main")
'''
!!!!!!!  Poner nombre del canal raiz!!!!!!
'''
root_channel = 'ungareaction'
'''
'''
path_channel = f'./data/{root_channel}'
print(f'Resultados en {path_channel}')
print(f'Descargando canal {root_channel}')
!python main.py --telegram-channel {root_channel} --output {path_channel}
# Convertir json en csv y excel y ordenar de mensaje más reciente a más antiguo
print(f'Construyendo el csv de mensajes de {root_channel}')
!python build-dataset.py --telegram-channel {root_channel} --data-path  {path_channel}
!head -1 {path_channel}/msgs_dataset.csv > {path_channel}/head_tmp.csv
!tail -n +2  {path_channel}/msgs_dataset.csv | sort -n -r -t "," --key=4 -o {path_channel}/tail_tmp.csv
!cat {path_channel}/head_tmp.csv {path_channel}/tail_tmp.csv | uniq > {path_channel}/msgs_dataset.csv
!rm {path_channel}/*tmp.*

Resultados en ./data/ungareaction
Descargando canal ungareaction

Init program at Sat May  4 11:45:25 2024


> Authorized!
created ./data/ungareaction

> Collecting data from Telegram Channel -> ungareaction
> ...

> Writing channel data...
created ./data/ungareaction/ungareaction/
> done.



> Writing posts data...
 put context: downloaded 628 messages since message 633
---> add new dataset to ./data/ungareaction
---> add new dataset to ./data/ungareaction
---> add new dataset to ./data/ungareaction
---> add new dataset to ./data/ungareaction
End program at Sat May  4 11:45:31 2024
Construyendo el csv de mensajes de ungareaction

Init program at Sat May  4 11:45:33 2024


 get context: last_msg: 633 , num_msg: 628
Reading data from channel -> ungareaction
Reading posts: 100% 628/628 [00:07<00:00, 80.99it/s]
-- END --




### Dado un canal descargado, extraer canales relacionados por método bola de nieve

Atención! El canal ha debido ser previamente extraido, ejecutando la anterior casilla


In [6]:
import os
import sys
# Establecer el directorio de trabajo
os.chdir("/content/drive/MyDrive/telegram-tracker-t-hoarder_tg-main")
'''
!!!!!!!  Poner nombre del canal raiz!!!!!!
'''
root_channel = 'ungareaction'
'''
'''
path_channel = f'./data/{root_channel}'
path_channel_n2 = f'./data/{root_channel}_n2'
with open(f'{path_channel}/related_channels.csv', 'r') as inputfile:
  channels = inputfile.readlines()
for channel in channels:
  channel = channel.strip('\n') # quitar salto de línea
  print(f'{channel}')
  !python main.py --telegram-channel {channel} --output {path_channel}_n2
  # convertir json en csv y excel
  !python build-dataset.py --telegram-channel {channel} --data-path  {path_channel}_n2


ungareaction

Init program at Sat May  4 11:47:16 2024


> Authorized!
created ./data/ungareaction_n2

> Collecting data from Telegram Channel -> ungareaction
> ...

> Writing channel data...
created ./data/ungareaction_n2/ungareaction/
> done.



> Writing posts data...
 put context: downloaded 628 messages since message 633
---> add new dataset to ./data/ungareaction_n2
---> add new dataset to ./data/ungareaction_n2
---> add new dataset to ./data/ungareaction_n2
---> add new dataset to ./data/ungareaction_n2
---> add new dataset to ./data/ungareaction_n2
---> add new dataset to ./data/ungareaction_n2
End program at Sat May  4 11:47:22 2024

Init program at Sat May  4 11:47:23 2024


 get context: last_msg: 633 , num_msg: 628
Reading data from channel -> ungareaction
Reading posts: 100% 628/628 [00:07<00:00, 82.14it/s]
-- END --

TheSpanishArmy

Init program at Sat May  4 11:47:33 2024


> Authorized!
created ./data/ungareaction_n2

> Collecting data from Telegram Channel -> TheSpanis


### Dado un conjunto de canales buscar sus relaciones

In [3]:
import os
import csv
# Establecer el directorio de trabajo
os.chdir("/content/drive/MyDrive/telegram-tracker-t-hoarder_tg-main")
'''
!!!!!!  Poner nombre del grupo de canales!!!!!!
'''
group = "ruido"
'''
'''
path_group = f'./data/{group}'
if not os.path.exists(f'{path_group}/list_channels.csv'):
  os.makedirs(path_group, exist_ok=True)
  channels = input('Enter a comma separated list of channels:    ')
  channels = channels.split (",")
  with open(f'{path_group}/list_channels.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter="\n")
    writer.writerow(channels)
else:
  with open(f'{path_group}/list_channels.csv', 'r') as inputfile:
    channels = inputfile.readlines()

for channel in channels:
  channel = channel.strip('\n') # quitar salto de línea
  print(f'{channel}')
  !python main.py --telegram-channel {channel} --output {path_group}
  # convertir json en csv y excel
  !python build-dataset.py --telegram-channel {channel} --data-path  {path_group}


Alviseperez

Init program at Sat May  4 11:41:30 2024


> Authorized!
created ./data/ruido
 get context: last_msg: 9985 , num_msg: 8862

> Collecting data from Telegram Channel -> Alviseperez
> ...

> Writing channel data...
created ./data/ruido/Alviseperez/
> done.



> Writing posts data...
 put context: downloaded 6 messages since message 9991
End program at Sat May  4 11:41:40 2024

Init program at Sat May  4 11:41:42 2024


 get context: last_msg: 9991 , num_msg: 6
Reading data from channel -> Alviseperez
Reading posts: 100% 6/6 [00:05<00:00,  1.06it/s]
-- END --

BeatrizTalegon

Init program at Sat May  4 11:41:53 2024


> Authorized!
created ./data/ruido
 get context: last_msg: 11160 , num_msg: 11020

> Collecting data from Telegram Channel -> BeatrizTalegon
> ...

> Writing channel data...
created ./data/ruido/BeatrizTalegon/
> done.



> Writing posts data...
 put context: downloaded 6 messages since message 11166
End program at Sat May  4 11:41:58 2024

Init program at Sat Ma

NameError: name 'sys' is not defined